#### Import Abstracts

In [ ]:
import pandas as pd
data = pd.read_csv("StanfordCo.csv")
data = data.drop('Unnamed: 0', axis=1)
data

,S,P,O,label,dboSource,Abstracts,Triples
0,http://yago-knowledge.org/resource/Van_Helsing...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,Van Helsing (film),Van_Helsing_(film),Van Helsing is a 2004 action gothic horror fil...,"[['is', '2004 action gothic horror film writte..."
1,http://yago-knowledge.org/resource/One_Flew_Ov...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,One Flew Over the Cuckoo's Nest (film),One_Flew_Over_the_Cuckoo's_Nest_(film),One Flew Over the Cuckoo's Nest is a 1975 Amer...,"[['is', '1975 American psychological comedy dr..."
2,http://yago-knowledge.org/resource/The_WB,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,The WB,The_WB,"The WB Television Network (for Warner Bros., o...","[['was', 'television network launched on broad..."
3,http://yago-knowledge.org/resource/This_Must_B...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,This Must Be the Place (film),This_Must_Be_the_Place_(film),This Must Be the Place is a 2011 Italian-Frenc...,"[['is directed', 'Sorrentino'], ['is Irish', '..."
4,"http://yago-knowledge.org/resource/Monsters,_Inc.",http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,"Monsters, Inc.","Monsters,_Inc.","Monsters, Inc. (also known as Monsters, Incorp...","[['is', '2001 animated monster comedy film for..."
...,...,...,...,...,...,...,...
12758,http://yago-knowledge.org/resource/Lincoln_Col...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,"Lincoln College, Oxford","Lincoln_College,_Oxford","Lincoln College (formally, The College of the ...","[['was founded by', 'then Bishop'], ['was foun..."
12759,http://yago-knowledge.org/resource/104th_Unite...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,104th United States Congress,104th_United_States_Congress,The 104th United States Congress was a meeting...,"[['was', 'meeting'], ['was', 'meeting of branc..."
12760,http://yago-knowledge.org/resource/London_Busi...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,London Business School,London_Business_School,London Business School (LBS) is a business sch...,"[['college of', 'federal University'], ['colle..."
12761,http://yago-knowledge.org/resource/Political_s...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,Political science,Political_science,Political science is the scientific study of p...,"[['is scientific study of', 'politics'], ['is ..."


#### Model

In [ ]:
from openie import StanfordOpenIE
from transformers import pipeline
import spacy

properties = {
    'openie.affinity_probability_cap': 1 / 3,
}

unmasker = pipeline('fill-mask', model='bert-base-uncased')
chunks = []
punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
item_list = ['he', 'his', 'it', 'she', 'her', 'they', 'them']
trips = []
triples = []
nlp = spacy.load("en_core_web_sm")
datatrips = []
j = 0
with StanfordOpenIE(properties=properties) as client:
    
    for s,p,o,label,dbo,abstract in data.values:

        if type(abstract) is float:
            datatrips.append([])
            print(j,label)
            j += 1
            continue

        print(j,label)


        #text = abstract
        #print('Text: %s.' % text)
        for triple in client.annotate(abstract):
            #print('|-', triple)
            triples.append(triple)

        for triple in triples:
            # turn triple into string 
            name = triples[0]['subject']
            string = (triple['subject'],triple['relation'],triple['object'])
            string = (' '.join(string))
            tokens = nlp(string)

            # if something in chunk
            if len(chunks) > 0:

                # if subject related to entity
                if any(good['subject'].casefold() == item for item in item_list) or good['subject'] in name or good['subject'] in label:
                    # if chunk is 'is' or 'was'
                    if chunks[-1] == 'is' or chunks[-1] == 'was':
                        ob = nlp(good['object'])

                        # get noun chunks from object
                        for b in ob.noun_chunks:
                            # make triple
                            slm = ("%s %s [MASK] %s" %(label, chunks[-1], b.text))
                            # check not already in triples
                            if ([unmasker(slm)[0]['token_str'], b.text]) not in trips:
                                trips.append([unmasker(slm)[0]['token_str'], b.text])

                    else:
                        ob = nlp(good['object'])

                        # get noun chunks from object
                        for b in ob.noun_chunks:
                            # mkae triple
                            slm = ("%s [MASK] %s %s" %(label, chunks[-1], b.text))
                            # if MASK is punctuation, remove MASK
                            if unmasker(slm)[0]['token_str'] in punc:
                                if ([chunks[-1], b.text]) not in trips:
                                    # check not already in triples
                                    trips.append([chunks[-1], b.text])

                            else:
                                if (['%s%s' %(unmasker(slm)[0]['token_str'], chunks[-1].capitalize()), b.text]) not in trips:
                                    # check not already in triples
                                    trips.append(['%s%s' %(unmasker(slm)[0]['token_str'], chunks[-1].capitalize()), b.text])

                # empty chunk list
                chunks = []

            # get chunk head 
            for chunk in tokens.noun_chunks:
                chunks.append(chunk.root.head.text)

            #print(triple)
            good = triple

        #for trip in trips:
            #print(trip)

        datatrips.append(trips)
        triples = []
        trips = []
        j += 1
    

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


0 Van Helsing (film)
Starting server with command: java -Xmx8G -cp /root/.stanfordnlp_resources/stanford-corenlp-4.5.3/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-110f43193aff4b53.props -preload openie


PermanentlyFailedException: ignored

In [ ]:
from openie import StanfordOpenIE
from transformers import pipeline
import spacy

properties = {
    'openie.affinity_probability_cap': 1 / 3,
}

unmasker = pipeline('fill-mask', model='bert-base-uncased')
chunks = []
punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
item_list = ['he', 'his', 'it', 'she', 'her', 'they', 'them']
trips = []
triples = []
nlp = spacy.load("en_core_web_sm")
datatrip = []
j = 0
with StanfordOpenIE(properties=properties) as client:
    
    for s,p,o,label,dbo,abstract in data.values:

        if type(abstract) is float:
            datatrip.append([])
            print(j,label)
            j += 1
            continue

        print(j,label)


        #text = abstract
        #print('Text: %s.' % text)
        for triple in client.annotate(abstract):
            #print('|-', triple)
            triples.append(triple)
            
        datatrip.append(trips)
        triples = []
        j += 1

In [ ]:
import claucy 
import spacy

properties = {
    'openie.affinity_probability_cap': 1 / 3,
}

#unmasker = pipeline('fill-mask', model='bert-base-uncased')
chunks = []
punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
item_list = ['he', 'his', 'it', 'she', 'her', 'they', 'them']
trips = []
triples = []
nlp = spacy.load("en_core_web_sm")
nlp_extract = spacy.load("en_core_web_sm")
nlp_sub = spacy.load("en_core_web_sm")
claucy.add_to_pipe(nlp) 
datatrip = []
j = 0
namest = 0
    
for s,p,o,label,dbo,abstract,_  in data.values:

    if type(abstract) is float:
        datatrip.append([])
        print(j,label)
        j += 1
        continue

    print(j,label)
    doc = nlp(abstract)

    for sent in doc.sents:

      #text = abstract
      #print('Text: %s.' % text)
      #doc_extract = nlp_extract(sent)  
          #print('|-', triple)
      if len(sent._.clauses) == 0:
        continue
      
      #print(sent._.clauses)
      propositions = sent._.clauses[0].to_propositions(as_text=False, inflect=None)  
      for triple in propositions:
        if namest == 0:
          name = str(triple[0])
          namest += 1
        if len(triple) == 3:
          sub = nlp_sub(str(triple[0]))
          tokenlist = []
          for token in sub:
            if token.pos_ == 'NOUN':
              tokenlist.append(token)
          if any(str(triple[0]).casefold() == str(item) for item in item_list) or any(str(item).casefold() in str(label).casefold() for item in tokenlist) or str(triple[0]) in name or str(triple[0]) in label:
            trips.append(triple)
    #for i in trips:
      #print(i)  
    #print()  
    datatrip.append(trips)
    namest = 0
    trips = []
    j += 1

Streaming output truncated to the last 5000 lines.
7763 Days of Thunder
7764 The Shipping News (film)
7765 George Lucas
7766 Tony Richardson
7767 Enosis Neon Paralimni FC
7768 Star Wars (film)
7769 Jennifer Morrison
7770 Minnesota Timberwolves
7771 Detroit
7772 Chronicle (film)
7773 Channel 5 (British TV channel)
7774 Mission: Impossible 2
7775 Jill Scott
7776 Elizabeth (film)
7777 Max Fleischer
7778 Jon Hamm
7779 Armin Mueller-Stahl
7780 Ghent
7781 Fred Zinnemann
7782 Finland
7783 Sinclair Broadcast Group
7784 Tim Bevan
7785 FC Spartak Moscow
7786 Whittier, California
7787 CeeLo Green
7788 Antigua and Barbuda national football team
7789 Daniel Nestor
7790 Tim Duncan
7791 La Toya Jackson
7792 The Practice
7793 Alan Turing
7794 The Royal Tenenbaums
7795 Suffolk
7796 The Nun's Story (film)
7797 Beijing Sinobo Guoan F.C.
7798 Dakar
7799 Alison Brie
7800 Golden Globe Award for Best Director
7801 City of Preston, Lancashire
7802 Hinduism
7803 Contemporary classical music
7804 Black comedy
7

In [ ]:
datatrip[:5]

[[(Van Helsing, is, a 2004 action gothic horror film),
  (It, stars, Hugh Jackman),
  (Van Helsing, is, tribute),
  (Van Helsing, is, a homage),
  (the film, Distributed, by Universal Pictures),
  (The film, received, negative reviews)],
 [(Flew, is, a 1975 American psychological comedy drama film),
  (It, features, a supporting cast),
  (Flew, Considered, by many to be one of the greatest films ever made),
  (The film, was, the second),
  (It, won, BAFTA Awards),
  (It, won, numerous Golden Globe),
  (the film, was deemed, by the United States Library of Congress),
  (the film, was deemed, In 1993),
  (the film,
   was deemed,
   culturally, historically, or aesthetically significant")],
 [(The WB Television Network, was, an American television network),
  (The WB Television Network, shut, on September 17, 2006),
  (The WB Television Network,
   shut,
   with some programs from both it and competitor UPN (which had shut down on September 15) moving to The CW when it launched the follo

In [ ]:
data['ClCyTriples'] = datatrip
data
#data.to_csv('Triplesdata.csv')

,S,P,O,label,dboSource,Abstracts,Triples,ClCyTriples
0,http://yago-knowledge.org/resource/Van_Helsing...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,Van Helsing (film),Van_Helsing_(film),Van Helsing is a 2004 action gothic horror fil...,"[['is', '2004 action gothic horror film writte...","[((Van, Helsing), (is), (a, 2004, action, goth..."
1,http://yago-knowledge.org/resource/One_Flew_Ov...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,One Flew Over the Cuckoo's Nest (film),One_Flew_Over_the_Cuckoo's_Nest_(film),One Flew Over the Cuckoo's Nest is a 1975 Amer...,"[['is', '1975 American psychological comedy dr...","[((Flew), (is), (a, 1975, American, psychologi..."
2,http://yago-knowledge.org/resource/The_WB,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,The WB,The_WB,"The WB Television Network (for Warner Bros., o...","[['was', 'television network launched on broad...","[((The, WB, Television, Network), (was), (an, ..."
3,http://yago-knowledge.org/resource/This_Must_B...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,This Must Be the Place (film),This_Must_Be_the_Place_(film),This Must Be the Place is a 2011 Italian-Frenc...,"[['is directed', 'Sorrentino'], ['is Irish', '...","[((This), (Be), (the, Place)), ((It), (stars),..."
4,"http://yago-knowledge.org/resource/Monsters,_Inc.",http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,"Monsters, Inc.","Monsters,_Inc.","Monsters, Inc. (also known as Monsters, Incorp...","[['is', '2001 animated monster comedy film for...","[((Monsters), is, (Inc.)), ((the, monster, wor..."
...,...,...,...,...,...,...,...,...
12758,http://yago-knowledge.org/resource/Lincoln_Col...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,"Lincoln College, Oxford","Lincoln_College,_Oxford","Lincoln College (formally, The College of the ...","[['was founded by', 'then Bishop'], ['was foun...","[((Lincoln, College, (, formally), is, (The, C..."
12759,http://yago-knowledge.org/resource/104th_Unite...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,104th United States Congress,104th_United_States_Congress,The 104th United States Congress was a meeting...,"[['was', 'meeting'], ['was', 'meeting of branc...","[((The, 104th, United, States, Congress), (was..."
12760,http://yago-knowledge.org/resource/London_Busi...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,London Business School,London_Business_School,London Business School (LBS) is a business sch...,"[['college of', 'federal University'], ['colle...","[((London, Business, School), is, (LBS)), ((th..."
12761,http://yago-knowledge.org/resource/Political_s...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,Political science,Political_science,Political science is the scientific study of p...,"[['is scientific study of', 'politics'], ['is ...","[((Political, science), (is), (the, scientific..."


In [ ]:

data1 = pd.read_csv("Triplesdata.csv")
data1 = data1.drop('Unnamed: 0', axis=1)
data1

In [ ]:
test = []
done = []
j = 0
for trips in data['ClCyTriples']:
  for trip in trips:
    if j == 0:
      print(["'%s'" %(trip[0]),"'%s'" %(trip[1]),"'%s'" %(trip[2])])
      j += 1
    test.append(["%s" %(trip[0]),"%s" %(trip[1]),"%s" %(trip[2])])

  done.append(test)
  test = []

done[:5]

["'Van Helsing'", "'is'", "'a 2004 action gothic horror film'"]


[[['Van Helsing', 'is', 'a 2004 action gothic horror film'],
  ['It', 'stars', 'Hugh Jackman'],
  ['Van Helsing', 'is', 'a homage'],
  ['Van Helsing', 'is', 'tribute'],
  ['the film', 'Distributed', 'by Universal Pictures'],
  ['The film', 'received', 'negative reviews']],
 [['Flew', 'is', 'a 1975 American psychological comedy drama film'],
  ['It', 'features', 'a supporting cast'],
  ['Flew', 'Considered', 'by many to be one of the greatest films ever made'],
  ['The film', 'was', 'the second'],
  ['It', 'won', 'numerous Golden Globe'],
  ['It', 'won', 'BAFTA Awards'],
  ['the film', 'was deemed', 'In 1993'],
  ['the film', 'was deemed', 'by the United States Library of Congress'],
  ['the film',
   'was deemed',
   'culturally, historically, or aesthetically significant"']],
 [['The WB Television Network', 'was', 'an American television network'],
  ['The WB Television Network',
   'shut',
   'with some programs from both it and competitor UPN (which had shut down on September 15) mo

In [ ]:
#data = data.drop('bbbb', axis=1)
data['ClCyTriples'] = done
#data
data.to_csv('another.csv')

In [ ]:
import pandas as pd
data = pd.read_csv('testing (1).csv')
#data1 = data1.drop('Unnamed: 0', axis=1)
len(data)

12763

In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 72.8 MB/s eta 0:00:00


In [ ]:
pip install git+https://github.com/mmxgn/spacy-clausie.git

In [ ]:
import spacy                                                                                                                                               
import claucy                                                                                                                                               
nlp = spacy.load("en_core_web_sm")
claucy.add_to_pipe(nlp)                                                                                                                                     
doc = nlp("AE died in Princeton in 1955.")   
print(doc._.clauses)                  
propositions = doc._.clauses[0].to_propositions(as_text=False, inflect=None)                                                                                               
for i in propositions:
  print(i)
  print(len(i))                         

[<SV, AE, died, None, None, None, [in Princeton, in 1955]>]
(AE, died, in 1955)
3
(AE, died, in Princeton, in 1955)
4
(AE, died, in Princeton)
3


In [ ]:
data = data.drop('Unnamed: 0', axis=1)
data

,S,P,O,label,dboSource,Abstracts,Triples,ClCyTriples
0,http://yago-knowledge.org/resource/Van_Helsing...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,Van Helsing (film),Van_Helsing_(film),Van Helsing is a 2004 action gothic horror fil...,"[['is', '2004 action gothic horror film writte...","[['Van Helsing', 'is', 'a 2004 action gothic h..."
1,http://yago-knowledge.org/resource/One_Flew_Ov...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,One Flew Over the Cuckoo's Nest (film),One_Flew_Over_the_Cuckoo's_Nest_(film),One Flew Over the Cuckoo's Nest is a 1975 Amer...,"[['is', '1975 American psychological comedy dr...","[['Flew', 'is', 'a 1975 American psychological..."
2,http://yago-knowledge.org/resource/The_WB,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,The WB,The_WB,"The WB Television Network (for Warner Bros., o...","[['was', 'television network launched on broad...","[['The WB Television Network', 'was', 'an Amer..."
3,http://yago-knowledge.org/resource/This_Must_B...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,This Must Be the Place (film),This_Must_Be_the_Place_(film),This Must Be the Place is a 2011 Italian-Frenc...,"[['is directed', 'Sorrentino'], ['is Irish', '...","[['This', 'Be', 'the Place'], ['It', 'stars', ..."
4,"http://yago-knowledge.org/resource/Monsters,_Inc.",http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,"Monsters, Inc.","Monsters,_Inc.","Monsters, Inc. (also known as Monsters, Incorp...","[['is', '2001 animated monster comedy film for...","[['Monsters', 'is', 'Inc.'], ['the monster wor..."
...,...,...,...,...,...,...,...,...
12758,http://yago-knowledge.org/resource/Lincoln_Col...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,"Lincoln College, Oxford","Lincoln_College,_Oxford","Lincoln College (formally, The College of the ...","[['was founded by', 'then Bishop'], ['was foun...","[['Lincoln College (formally', 'is', 'The Coll..."
12759,http://yago-knowledge.org/resource/104th_Unite...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,104th United States Congress,104th_United_States_Congress,The 104th United States Congress was a meeting...,"[['was', 'meeting'], ['was', 'meeting of branc...","[['The 104th United States Congress', 'was', '..."
12760,http://yago-knowledge.org/resource/London_Busi...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,London Business School,London_Business_School,London Business School (LBS) is a business sch...,"[['college of', 'federal University'], ['colle...","[['London Business School', 'is', 'LBS'], ['th..."
12761,http://yago-knowledge.org/resource/Political_s...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,Political science,Political_science,Political science is the scientific study of p...,"[['is scientific study of', 'politics'], ['is ...","[['Political science', 'is', 'the scientific s..."


In [ ]:
from transformers import pipeline
import spacy
import ast

unmasker = pipeline('fill-mask', model='bert-base-uncased', top_k=1, framework='pt', device = -1)
chunks = []
punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
item_list = ['he', 'his', 'it', 'she', 'her', 'they', 'them']
trips = []
triples = []
nlp = spacy.load("en_core_web_sm")
datatrips = []
j = 0

for s,p,o,label,dbo,abstracts,triples,_,_ in data.values:
    
  if type(abstracts) is float:
    datatrips.append([])
    print(j,label, '- nothing')
    j += 1
    continue

  if j % 1000 == 0:          
    print(j,label)
    
  for triple in ast.literal_eval(triples):
      #triple = ast.literal_eval(triple)
      # turn triple into string 
      string = (label,triple[0],triple[1])
      string = (' '.join(string))
      tokens = nlp(string)
      #print(string)

      for chunk in tokens.noun_chunks:
          chunks.append(chunk.root.head.text)

      # if something in chunk
      if len(chunks) > 0:
          #print('chunk',chunks[-1])
          if chunks[-1] == 'is' or chunks[-1] == 'was':
              ob = nlp(str(triple[1]))
              #print('ob',ob)
              # get noun chunks from object
              for b in ob.noun_chunks:
                #print('b',b)
                # make triple
                slm = ("%s %s [MASK] %s" %(label, chunks[-1], b.text))
                # check not already in triples
                if ([unmasker(slm)[0]['token_str'], b.text]) not in trips:
                    trips.append([unmasker(slm)[0]['token_str'], b.text])

          else:
              ob = nlp(str(triple[1]))
              #print('ob',ob)
              # get noun chunks from object
              for b in ob.noun_chunks:
                #print('b',b)
                # mkae triple
                slm = ("%s [MASK] %s %s" %(label, chunks[-1], b.text))
                # if MASK is punctuation, remove MASK
                if unmasker(slm)[0]['token_str'] in punc:
                    if ([chunks[-1], b.text]) not in trips:
                      # check not already in triples
                      trips.append([chunks[-1], b.text])

                else:
                  if (["%s%s" %(unmasker(slm)[0]['token_str'], chunks[-1].capitalize()),b.text]) not in trips:
                    # check not already in triples
                    trips.append(["%s%s" %(unmasker(slm)[0]['token_str'], chunks[-1].capitalize()),b.text])
      else:
        print(j, label, '- different')
        #slm = ("%s [MASK] %s" %(label, b.text))
        trips.append([str(triple[0]),triple[1]])
          # empty chunk list
      chunks = []

        # get chunk head 
        #for chunk in tokens.noun_chunks:
            #chunks.append(chunk.root.head.text)

        #print(triple)
        #good = triple

  #for trip in trips:
      #print(trip)

  datatrips.append(trips)
  trips = []
  j += 1

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


0 Van Helsing (film)
7 Silkeborg IF - different
7 Silkeborg IF - different
33 Armando Iannucci - different
38 F.C. Hansa Rostock - nothing
61 Unforgiven - different
61 Unforgiven - different
61 Unforgiven - different
61 Unforgiven - different
142 Normandy - different
146 Bedazzled (2000 film) - different
146 Bedazzled (2000 film) - different
169 Atonement (film) - nothing


In [ ]:
from google.colab import files
data['Processed-Trips-Stanford'] = datatrips
data.to_csv('final-Stanford-Trips.csv')
files.download('final-Stanford-Trips.csv')

In [ ]:
for i in data['Processed-Trips-ClCy'][5:15]:
  for j in i:
    print(j)
  print()

NameError: ignored

# final processing 

In [2]:
import pandas as pd
clcy = pd.read_csv("final-Clcy-Trips.csv")
clcy = clcy.drop('Unnamed: 0', axis=1)
clcy

,S,P,O,label,dboSource,Abstracts,Triples,ClCyTriples,Processed-Trips-ClCy
0,http://yago-knowledge.org/resource/Van_Helsing...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,Van Helsing (film),Van_Helsing_(film),Van Helsing is a 2004 action gothic horror fil...,"[['is', '2004 action gothic horror film writte...","[['Van Helsing', 'is', 'a 2004 action gothic h...","[['also', 'a 2004 action gothic horror film'],..."
1,http://yago-knowledge.org/resource/One_Flew_Ov...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,One Flew Over the Cuckoo's Nest (film),One_Flew_Over_the_Cuckoo's_Nest_(film),One Flew Over the Cuckoo's Nest is a 1975 Amer...,"[['is', '1975 American psychological comedy dr...","[['Flew', 'is', 'a 1975 American psychological...","[['also', 'a 1975 American psychological comed..."
2,http://yago-knowledge.org/resource/The_WB,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,The WB,The_WB,"The WB Television Network (for Warner Bros., o...","[['was', 'television network launched on broad...","[['The WB Television Network', 'was', 'an Amer...","[['formerly', 'an American television network'..."
3,http://yago-knowledge.org/resource/This_Must_B...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,This Must Be the Place (film),This_Must_Be_the_Place_(film),This Must Be the Place is a 2011 Italian-Frenc...,"[['is directed', 'Sorrentino'], ['is Irish', '...","[['This', 'Be', 'the Place'], ['It', 'stars', ...","[['mustBe', 'the Place'], ['stars', 'Sean Penn..."
4,"http://yago-knowledge.org/resource/Monsters,_Inc.",http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,"Monsters, Inc.","Monsters,_Inc.","Monsters, Inc. (also known as Monsters, Incorp...","[['is', '2001 animated monster comedy film for...","[['Monsters', 'is', 'Inc.'], ['the monster wor...","[['monsters', 'Inc.'], ['at', 'the El Capitan ..."
...,...,...,...,...,...,...,...,...,...
12758,http://yago-knowledge.org/resource/Lincoln_Col...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,"Lincoln College, Oxford","Lincoln_College,_Oxford","Lincoln College (formally, The College of the ...","[['was founded by', 'then Bishop'], ['was foun...","[['Lincoln College (formally', 'is', 'The Coll...","[['also', 'The College'], ['portraitOf', 'Rich..."
12759,http://yago-knowledge.org/resource/104th_Unite...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,104th United States Congress,104th_United_States_Congress,The 104th United States Congress was a meeting...,"[['was', 'meeting'], ['was', 'meeting of branc...","[['The 104th United States Congress', 'was', '...","[['not', 'a meeting'], ['from', 'January'], ['..."
12760,http://yago-knowledge.org/resource/London_Busi...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,London Business School,London_Business_School,London Business School (LBS) is a business sch...,"[['college of', 'federal University'], ['colle...","[['London Business School', 'is', 'LBS'], ['th...","[['in', 'LBS'], ['hasAcquired', 'the Marylebon..."
12761,http://yago-knowledge.org/resource/Political_s...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,Political science,Political_science,Political science is the scientific study of p...,"[['is scientific study of', 'politics'], ['is ...","[['Political science', 'is', 'the scientific s...","[['also', 'the scientific study'], ['not', 'a ..."


In [ ]:
import ast
#k = 0
#for i in ast.literal_eval(clcy['Processed-Trips-ClCy']):
  #print(i)
  #if k == 10:
    #break
  #k += 1
  #print()
#ast.literal_eval(clcy['Processed-Trips-ClCy'])
ast.literal_eval(clcy['Processed-Trips-ClCy'][0])

[['also', 'a 2004 action gothic horror film'],
 ['stars', 'Hugh Jackman'],
 ['also', 'a homage'],
 ['producedBy', 'Universal Pictures'],
 ['hasReceived', 'negative reviews']]

In [ ]:
final_trips = []
import nltk
import ast
trips = []
for s,p,o,label,dbo,abstract,_,_,triples in clcy.values:
  if type(abstract) == float:
    final_trips.append([])
    continue
    
  for triple in ast.literal_eval(triples):
    if triple[0] == 'is' or triple[0] == 'also' or triple[0] == 'an' or triple[0] == 'a' or triple[0] == 'was':
      simpletok = triple[1].split(' ')
      if simpletok[0] == 'the' or simpletok[0] == 'a' or simpletok[0] == 'an':
        print(triple[0],'- type'," ".join(simpletok[1:]))
        trips.append(['type'," ".join(simpletok[1:])])
      else:
        print(triple[0], '-', 'type', triple[1])
        trips.append(['type', triple[1]])
    else:
      simpletok = triple[1].split(' ')
      if simpletok[0] == 'the' or simpletok[0] == 'a' or simpletok[0] == 'an':
        print(triple[0]," ".join(simpletok[1:]))
        trips.append([triple[0], " ".join(simpletok[1:])])
      else:
        trips.append(triple)
  final_trips.append(trips)
  trips = []


In [4]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [5]:
trips = []
done = []
nouns = []
import spacy 
nlp = spacy.load("en_core_web_sm")
#for s,p,o,label,dbo,abstract,_,_,triples in clcy.values:
  #if type(abstract) == float:
    #done.append([])
    #continue
for triples in final_trips: 
  for triple in triples:
    dd = triple[0].split(' ')
    if len(dd) > 1:
      #print(triple[0], '-', label, triple[0].split(' ')[1], triple[1])
      tokens = nlp(triple[1])
      for token in tokens.noun_chunks:
        nouns.append(token)
      if len(nouns) == 1:
        print(triple[0], '-', label, triple[0].split(' ')[1], nouns[0])
        trips.append([triple[0].split(' ')[1], nouns[0]])
      else:
        tokenized = nltk.word_tokenize(triple[1])
        pos = nltk.pos_tag(tokenized)
        for i in pos:
          if i[1] == 'CD' or i[1] == 'NN':
            print(triple[0], '-', label, triple[0].split(' ')[1], i[0])
            trips.append([triple[0].split(' ')[1], i[0]])
      nouns = []
    else:
      trips.append(triple)
  done.append(trips)
  trips = []

#('type'.split(' '))

Being declared - Martin Sheen declared one
was replaced - Martin Sheen replaced 2022
was founded - Martin Sheen founded 1980
had grossed - Martin Sheen grossed million
was ranked - Martin Sheen ranked tenth
was voted - Martin Sheen voted best Scottish film


In [6]:
done[:5]

[[['type', '2004 action gothic horror film'],
  ['stars', 'Hugh Jackman'],
  ['type', 'homage'],
  ['producedBy', 'Universal Pictures'],
  ['hasReceived', 'negative reviews']],
 [['type', '1975 American psychological comedy drama film'],
  ['features', 'supporting cast'],
  ['oneOf', 'greatest films'],
  ['won', 'BAFTA Awards'],
  ['won', 'numerous Golden Globe'],
  ['stateOf', 'United States Library'],
  ['libraryOf', 'Congress']],
 [['formerly', 'American television network'],
  ['launchedOn', 'September'],
  ['alsoLaunched', 'some programs'],
  ['alsoLaunched', 'both it'],
  ['launched', 'competitor UPN'],
  ['wasLaunched', 'which'],
  ['wasLaunched', 'September'],
  ['alsoLaunched', 'The CW'],
  ['thenLaunched', 'it'],
  ['wasLaunched', 'following day'],
  ['closedIn', 'December']],
 [['mustBe', 'Place'],
  ['stars', 'Sean Penn'],
  ['stars', 'Frances McDormand'],
  ['isIn', 'middle-aged wealthy rock star'],
  ['isIn', 'who'],
  ['spentIn', 'his retirement'],
  ['involvedIn', 'ques

In [11]:
final_trips[:5]

[[['type', '2004 action gothic horror film'],
  ['stars', 'Hugh Jackman'],
  ['type', 'homage'],
  ['producedBy', 'Universal Pictures'],
  ['hasReceived', 'negative reviews']],
 [['type', '1975 American psychological comedy drama film'],
  ['features', 'supporting cast'],
  ['oneOf', 'greatest films'],
  ['won', 'BAFTA Awards'],
  ['won', 'numerous Golden Globe'],
  ['stateOf', 'United States Library'],
  ['libraryOf', 'Congress']],
 [['formerly', 'American television network'],
  ['launchedOn', 'September'],
  ['alsoLaunched', 'some programs'],
  ['alsoLaunched', 'both it'],
  ['launched', 'competitor UPN'],
  ['wasLaunched', 'which'],
  ['wasLaunched', 'September'],
  ['alsoLaunched', 'The CW'],
  ['thenLaunched', 'it'],
  ['wasLaunched', 'following day'],
  ['closedIn', 'December']],
 [['mustBe', 'Place'],
  ['stars', 'Sean Penn'],
  ['stars', 'Frances McDormand'],
  ['isIn', 'middle-aged wealthy rock star'],
  ['isIn', 'who'],
  ['spentIn', 'his retirement'],
  ['involvedIn', 'ques

In [10]:
trips = []
final = []
for triples in done:
  for triple in triples:
    obj = str(triple[1]).split(" ")
    obj = "_".join(obj)
    trips.append([triple[0], obj])
  final.append(trips)
  trips = []
len(final), final[:5]

(12763,
 [[['type', '2004_action_gothic_horror_film'],
   ['stars', 'Hugh_Jackman'],
   ['type', 'homage'],
   ['producedBy', 'Universal_Pictures'],
   ['hasReceived', 'negative_reviews']],
  [['type', '1975_American_psychological_comedy_drama_film'],
   ['features', 'supporting_cast'],
   ['oneOf', 'greatest_films'],
   ['won', 'BAFTA_Awards'],
   ['won', 'numerous_Golden_Globe'],
   ['stateOf', 'United_States_Library'],
   ['libraryOf', 'Congress']],
  [['formerly', 'American_television_network'],
   ['launchedOn', 'September'],
   ['alsoLaunched', 'some_programs'],
   ['alsoLaunched', 'both_it'],
   ['launched', 'competitor_UPN'],
   ['wasLaunched', 'which'],
   ['wasLaunched', 'September'],
   ['alsoLaunched', 'The_CW'],
   ['thenLaunched', 'it'],
   ['wasLaunched', 'following_day'],
   ['closedIn', 'December']],
  [['mustBe', 'Place'],
   ['stars', 'Sean_Penn'],
   ['stars', 'Frances_McDormand'],
   ['isIn', 'middle-aged_wealthy_rock_star'],
   ['isIn', 'who'],
   ['spentIn', 'his

In [57]:
trips = []
finalfinal = []
for triples in final:
  for triple in triples:
    if triple[0] == '|' or triple[0] == '"':
      print('ye -', triple)
    elif '...' in triple[0] or '2010' in triple[0] or '=' in triple[0] or triple[0] == "/'" or triple[0] == '¹' or triple[0] == '2002-' or triple[0] == 'R&' or triple[0] == "'" or triple[0] == '##' or triple[0] == 'alsoStarred-1989->' or triple[0] == '[UNK]' or triple[0] == '1972-' or triple[0] == 'ᴬ' or triple[0] == 'Hawai' or triple[0] == "universityHawai'" or triple[0] == '«' or triple[0] == '2005-' or triple[0] == '1964-' or triple[0] == '2007-' or triple[0] == '+' or triple[0] == "university/'" or triple[0] == '2003-' or triple[0] == 'guestStarred-1989-':
      print('working -',triple)

    else:
      trips.append(triple)
    
  finalfinal.append(trips)
  trips = []

len(finalfinal), final[:5]    

working - ['[UNK]', '深作']
working - ['[UNK]', '埼玉県']
working - ['[UNK]', '静岡県']
working - ['[UNK]', '能登']
ye - ['"', 'strategically_important_"Gateway']
working - ['[UNK]', '日本語']
working - ['[UNK]', 'སྐུ་མདུན་_Wylie']
working - ['2010),Joe', 'Mandy']
working - ['[UNK]', 'Giles']
working - ['[UNK]', '鹿児島県']
ye - ['"', '"science_team_collaborator']
working - ['[UNK]', '中日ドラゴンズ']
working - ['[UNK]', '白灵']
working - ['[UNK]', '埼玉西武ライオンズ']
working - ['[UNK]', '坂本_真綾']
working - ['[UNK]', '林原_めぐみ']
working - ['=Used', 'record']
working - ['[UNK]', '広島県']
working - ['[UNK]', 'Tuhuⱡnana']
working - ['[UNK]', '愛知県']
working - ['[UNK]', '機動戦士ガンダム00（ダブルオー']
working - ['[UNK]', '神谷_明']
working - ['[UNK]', 'ڪراچي']
working - ['...In', "AFI's_100_Years...100_Passions_survey"]
working - ['[UNK]', 'ජාතික_පක්ෂය']
working - ['...Becomes', 'What']
working - ['[UNK]', '宮崎_駿']
ye - ['|', 'ə']
working - ['[UNK]', '株式会社エニックス']
working - ['[UNK]', 'ˈɦɔlɑnt']
working - ['[UNK]', '最優秀監督賞']
working - ['[UNK]', 

(12763,
 [[['type', '2004_action_gothic_horror_film'],
   ['stars', 'Hugh_Jackman'],
   ['type', 'homage'],
   ['producedBy', 'Universal_Pictures'],
   ['hasReceived', 'negative_reviews']],
  [['type', '1975_American_psychological_comedy_drama_film'],
   ['features', 'supporting_cast'],
   ['oneOf', 'greatest_films'],
   ['won', 'BAFTA_Awards'],
   ['won', 'numerous_Golden_Globe'],
   ['stateOf', 'United_States_Library'],
   ['libraryOf', 'Congress']],
  [['formerly', 'American_television_network'],
   ['launchedOn', 'September'],
   ['alsoLaunched', 'some_programs'],
   ['alsoLaunched', 'both_it'],
   ['launched', 'competitor_UPN'],
   ['wasLaunched', 'which'],
   ['wasLaunched', 'September'],
   ['alsoLaunched', 'The_CW'],
   ['thenLaunched', 'it'],
   ['wasLaunched', 'following_day'],
   ['closedIn', 'December']],
  [['mustBe', 'Place'],
   ['stars', 'Sean_Penn'],
   ['stars', 'Frances_McDormand'],
   ['isIn', 'middle-aged_wealthy_rock_star'],
   ['isIn', 'who'],
   ['spentIn', 'his

In [49]:
clcy['ClCy'] = finalfinal

In [50]:
clcy.to_csv('clcyfinal.csv')

In [ ]:
pip install rfc3987

In [103]:
import rdflib
from rdflib import Graph
from rdflib.namespace import RDF
from rdflib import Literal, XSD
flag = True
g = Graph()
k = 0
n = 1
for s,p,o,label,dbo,abstract,_,_,_,triples in clcy.values:
  #print(k,label)
  if type(abstract) == float:
    #print('nothing')
    k+=1
    continue
  for triple in triples:
    flag = True
    try:
      int(triple[1])
    except:
      flag = False 
    
    if flag is True:
      rel = triple[0].replace("'", '')
      rel = rel.replace('"', '')
      rel = rel.replace('`', '')
      rel = rel.replace('.', '')
      schema = ('http://schema.org/%s' %(rel))
      year = int(triple[1])
      year = Literal(year, datatype=XSD.gYear)
      try:
        parse(schema, rule='IRI')
      except:
        print(n,triple)
        n+=1
        continue
      g.add( (rdflib.URIRef(s), rdflib.URIRef(schema), year ))
    else:
      obj = triple[1].replace("'", '')
      obj = obj.replace('"', '')
      obj = obj.replace('`', '')
      obj = obj.replace('.', '')
      link = 'http://yago-knowledge.org/resource/%s' %(obj)
      if triple[0] == 'type':
        try:
          parse(link, rule='IRI')
        except:
          print(n,triple)
          n+=1
          continue
        g.add( (rdflib.URIRef(s), RDF.type, rdflib.URIRef(link)) )
      else:
        rel = triple[0].replace("'", '')
        rel = rel.replace('"', '')
        rel = rel.replace('`', '')
        rel = rel.replace('.', '')
        schema = ('http://schema.org/%s' %(rel))
        try:
          parse(link, rule='IRI')
        except:
          print(n,triple)
          n+=1
          continue
        try:
          parse(schema, rule='IRI')
        except:
          print(n,triple)
          n+=1
          continue
        g.add( (rdflib.URIRef(s), rdflib.URIRef(schema), rdflib.URIRef(link)) )
  k+=1

1 ['##zAs', "Irv_Gotti's_Murder_Inc._Records"]
2 ['##cSalt', 'salt']
3 ['##sCombines', 'new_wave']
4 ['##eIn', 'science_fiction']
5 ['##nessOf', 'preservation']
6 ['##nessOf', 'Library']
7 ['##landPlayed', 'Wimbledon']
8 ['##landPlayed', 'Lincoln_City']
9 ['##landPlayed', 'Newcastle_United']
10 ['##landPlayed', 'Wolverhampton_Wanderers']
11 ['##landPlayed', 'Leicester_City']
12 ['##oPlayed', 'Spanish_club_UD_Marbella']
13 ['##landPlayed', 'Norwich_City']
14 ['##landPlayed', 'Brentford']
15 ['##landPlayed', 'North_American_Soccer_League_side']
16 ['##landPlayed', 'Tampa_Bay_Rowdies']
17 ['##landIn', 'England']
18 ['##sKielce', 'Katowice']
19 ['##sKielce', 'Kielce']
20 ['##sKielce', 'Krosno_Voivodeships']
21 ['##yOn', 'AMC_drama_series']
22 ['##ingPlayed', 'Jean_Ward']
23 ['##sDirect', 'condition']
24 ['##iAs', 'Ava_Alexander']
25 ['##sAs', 'Oliver_Wendell_Douglas']
26 ['##sSitcom', 'Green_Acres']
27 ['##aPlayed', 'long-running_role']
28 ['##aVoiced', 'Diane_Simmons']
29 ['##sMeet', 'Bar

2307/3669470 triples were not in proper format and needed to be deleted

In [106]:
total = 0
for triples in clcy['Processed-Trips-ClCy']:
  if len(triples) == 0:
    continue 
  else:
    for triple in triples:
      total += 1
total

3669470

In [104]:
g.serialize(destination='clcyfinal.ttl', format='turtle')

<Graph identifier=N8e17c02378df4be390d5b9d888080301 (<class 'rdflib.graph.Graph'>)>